# APIM ❤️ OpenAI

## Function calling lab
![flow](../../images/function-calling.gif)

Playground to try the OpenAI [function calling](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython) feature with an Azure Functions API also managed with APIM. 

At a high level you can break down working with functions into three steps:
1. Call the chat completions API with your functions and the user’s input
2. Use the model’s response to call your API or function
3. Call the chat completions API again, including the response from your function to get a final response

### Result
![result](result.png)

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [4️⃣ Deploy the function](#4)
- [🧪 Test the Function API](#function)
- [🧪 Test OpenAI function calling](#functioncalling)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [Azure Functions Core Tools](https://learn.microsoft.com/en-us/azure/azure-functions/functions-run-local?tabs=windows%2Cisolated-process%2Cnode-v4%2Cpython-v2%2Chttp-trigger%2Ccontainer-apps&pivots=programming-language-python#install-the-azure-functions-core-tools) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [9]:
import os
import json
import datetime
import requests

lab_prefix="aif-pevo-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
#apim_resource_sku = "Basicv2"
apim_resource_sku = "Consumption" 
openai_resources = [ {"name": "openai1", "location": "swedencentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "1106"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-03-01-preview"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/2024-02-01/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ ]

log_analytics_name = lab_prefix + "workspace"
app_insights_name = lab_prefix + 'insights'

function_app_name = lab_prefix + "function"
#storage_account_name = (lab_prefix + "storage").replace('-','')[:20]
storage_account_name='storage'
print("storage_account_name",storage_account_name)


storage_account_name storage


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [10]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  aif-pevo-function-calling  created ⌚  16:11:07.237222


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [11]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "functionAppName": { "value": function_app_name },
    "storageAccountName": { "value": storage_account_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


{
  "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aif-pevo-function-calling/providers/Microsoft.Resources/deployments/function-calling",
  "location": null,
  "name": "function-calling",
  "properties": {
    "correlationId": "461c9e98-5e21-4396-861a-9ff806985964",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aif-pevo-function-calling/providers/Microsoft.ApiManagement/service/apim-m4z5osvxasqhi",
            "resourceGroup": "aif-pevo-function-calling",
            "resourceName": "apim-m4z5osvxasqhi",
            "resourceType": "Microsoft.ApiManagement/service"
          }
        ],
        "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aif-pevo-function-calling/providers/Microsoft.ApiManagement/service/apim-m4z5osvxasqhi/apis/openai",
        "resourceGroup": "aif-pevo-function-calling",
        "r


c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\function-calling\main.bicep(319,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]

c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\function-calling\main.bicep(336,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]

c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\function-calling\main.bicep(547,12) : Warning BCP036: The property "value" expected a value of type "null | string" but the provided value is of type "object". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-

618

<a id='3'></a>
### 3️⃣ Get the deployment outputs


In [ ]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.functionAppResourceName.value -o tsv
function_app_resource_name = deployment_stdout.n
print("👉🏻 Function Name: ", function_app_resource_name)


<a id='4'></a>
### 4️⃣ Deploy the function

Deploy the local function project to the function app resource previously created

In [ ]:
! func azure functionapp publish {function_app_resource_name}

<a id='function'></a>
### 🧪 Test the Function API

In [ ]:
request = { "location": "London", "unit": "celsius" }
url = apim_resource_gateway_url + "/weather"
response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = request)
if (response.status_code == 200):
    data = json.loads(response.text)
    print("location: ", data.get("location"))
    print("unit: ", data.get("unit"))
    print("temperature: ", data.get("temperature"))
else:
    print(response.text)


<a id='functioncalling'></a>
### 🧪 Test OpenAI function calling

The following code was reused from the [Azure OpenAI documentation](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling?tabs=non-streaming%2Cpython). The `UUID` will be used to search for the end-to-end transaction in Application insights.

- Note: run ```pip install openai``` in a terminal before executing this step.

In [ ]:
from openai import AzureOpenAI
import uuid

prompt = "What's the weather like in London and its big sister cities?"

UUID = str(uuid.uuid4())
print(f"Request-Id: {UUID} - use this ID to trace the requests in Azure Application Insights.")

client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)

# Example function hard coded to return the same weather
# the local function calls the API
def get_current_weather(location, unit):
    request = { "location": location, "unit": unit }
    url = apim_resource_gateway_url + "/weather"
    response = requests.post(url, headers = {'api-key':apim_subscription_key, "Request-Id": UUID}, json = request)
    if (response.status_code == 200):
        return response.text
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

# Step 1: send the conversation and available functions to the model
messages = [{"role": "user", "content": prompt}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location, using the local temperature measuring unit.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The temperature measuring unit, e.g. celsius for London or fahrenheit for US cities",
                        "enum": ["celsius", "fahrenheit"]
                    },
                },
                "required": ["location", "unit"],
            },
        },
    }
]
print("▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt\n", prompt)
response = client.chat.completions.create(
    model=openai_deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
    extra_headers = {"Request-Id": UUID}
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
if tool_calls:
    # Step 2: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    messages.append(response_message)  # extend conversation with assistant's reply
    # send the info for each function call and function response to the model
    print("▶️ Step 2: call the functions")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        print(function_response)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response
    print("▶️ Step 3: finish with a completion to anwser the user prompt using the function response")
    second_response = client.chat.completions.create(
        model=openai_deployment_name,
        messages=messages,
        extra_headers = {"Request-Id": UUID}
    )  # get a new response from the model where it can see the function response
    print(second_response.choices[0].message.content)



<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.